написать класс для запросов

Do not contact the server more often than once every 10 seconds.\
Do not poll for any single RID more often than once a minute.\
Use the URL parameter email and tool, so that the NCBI can contact you if there is a problem.\
Run scripts weekends or between 9 pm and 5 am Eastern time on weekdays if more than 50 searches will be submitted.


If your queries are short (less than a few hundred bases) we suggest you merge them into one search of up to 1,000 bases


# пример - https://github.com/biopython/biopython/blob/master/Bio/Blast/NCBIWWW.py

# Добавить
 ## поиск taxid по названию (выбирать по умолчанию один самый совпадающий?)
 https://ena-docs.readthedocs.io/en/latest/faq/taxonomy.html тут можно посмотреть удачный способ это сделать
 
 ## вещи которые мог менять древний апи бласта 
 https://ncbi.github.io/blast-cloud/dev/api.html

In [404]:
import urllib
import urllib3
import requests
import time
import re

from bs4 import BeautifulSoup

Пример использования:

    - последовательность: 
        "MVMFGNKVSYPYMDLSPKSHWIVYEGNLKFYDKQGRDTASIKDAASLSFDFFTWGRNPKVAMIETTEVSKLSVESFKSTCYGYIICSYT",
    - база данных: Whole_Genome_Shotgun_contigs,
    - организм: Prevotella sp. oral taxon 472 str. F0295 (taxid:619693),
    - tblastn.

In [268]:
# поиск организма в базе данных WGS https://www.ncbi.nlm.nih.gov/Traces/wgs/

In [269]:
WGS_PROJECTS_URL = 'https://www.ncbi.nlm.nih.gov/Traces/wgs/index.cgi?'

In [270]:
WGS_payload = {
}

text:*Prevotellaceae* AND project_s:wgs AND (project_s:WGS) AND (project_s:WGS)

In [ ]:
# генерация уникального имени - https://docs.python.org/3/library/uuid.html
import uuid

# make a random UUID
uuid.uuid4()

# make a UUID from a string of hex digits (braces and hyphens ignored)
x = uuid.UUID('{00010203-0405-0607-0809-0a0b0c0d0e0f}')

# рабочий простой вариант

# запрос в ncbi

In [271]:
fasta = \
""">line1
MVMFGNKVSYPYMDLSPKSHWIVYEGNLKFYDKQGRDTASIKDAASLSFDFFTWGRNPKVAMIETTEVSKLSVESFKSTCYGYIICSYT
"""

In [343]:
fasta = \
""">sf
MVMFGNKVSYPYMDLSPKSHWIVYEGNLKFYDKQGRDTASIKDAASLSFDFFTWGRNPKVAMIETTEVSKLSVESFKSTCYGYIICSYT
>sdf
MKEDNCMMFIKVLFTLMLVTMVSAGCKGHTGRPKSKTDIDISRLELIKVKEKEIRQYYASWCDVGLDGIAVMPTEQQERYTDIAIELLHEALSKAGYKPVTNQKAAAAIRKYFNVDITEKNCLLHRYGIHRYVCKSEAINDRDAQKKLMSPEVDEADDFWDLQIFVPNYNYIVYHPTIDQAVQIDGFNLEDCDLNNIKADKLSYRLDKSILYQNLFIFYDSKAAFVWLKANNPSFLTCLF
>sdfs
MKEDNCMMFIKVLFTLMLVTMVSAGCKGHTGRPKSKTDIDISRLELIKVKEKEIRQYYASWCDVGLDGIAVMPTVLFTLMLVTLLHEALSKAGVLFTLMLVTNVDITEKNCLLHRYGIHRYVCKSEAINDRDAQKKLMSPEVDEADDFWDLQIFVPNYNYIVYHPTIDQAVQIDGFNLEDCDLNNIKADKLSYRLDKSILYQNLFIFYDSKAAFVWLKANNPSFLTCLF
>dfgdfgd
KLSFDFAKSLSEVEPIPHIDRSISPIVNLYYRGGKYYLESPFVSNSEGKENVK
>dfgdf
KLSFDFPIPHIDRSISPIAKSLSEVEPIPHIDRSISPIVNLYYRGGKYYLESPFVSNSEGKENVKPIPHIDRSISPI
>dvsv
LMLVTMVSAGCKGHTGRPKSKTDIDISRLELIKVKEKEIRQYYASWCDVGLDGIAVMPTEQQERYTDIAIELLHEALSKAGYKPVTNQKAAAAIRKYFNVDITEKNCLLHRYGIHRYVCKSEAINDRDAQKKLMSPEVDEADDFWDLQIFVPNYNYIVYHPTIDQAVQIDGFNLEDCDLNNIKADKLSYRLDKSILYQNLFIF
>dvssd
LMLVTMVSAGCKGHTGRPKSKTDIDISRLELIKVKEKLMLVTEIRQYYASWCDVGLDGIAVMPTEQQERYTDILMLVTAIELLHEALSKAGYKPVTNQKAAAAILMLVTRKYFNVDITEKNCLLHRYGIHRYVCKSEAINDRDAQKKLMSPEVDEADDFWDLQIFVPNYNYIVYHPTIDQAVQIDGFNLEDCDLNNIKADKLSYRLDKSILYQNLFIF
"""

In [344]:
BLAST_URL = "https://blast.ncbi.nlm.nih.gov/Blast.cgi"

In [345]:
payload = {
    'QUERY': fasta,
    'db': 'protein',
    #'QUERYFILE': '(binary)',  # use for binary files
    'GENETIC_CODE': 1,
    'JOB_TITLE': 'fasta',
    'ADV_VIEW': 'true',
    'stype': 'nucleotide',
    #'SUBJECTFILE': '(binary)', # use for binary files
    'DATABASE': 'Whole_Genome_Shotgun_contigs',
    'DB_GROUP': 'wgsOrg',
    #'EQ_MENU': 'Prevotella sp. oral taxon 472 str. F0295 (taxid:619693)',  # работает если передавать полное название из выпадающего списка
    'EQ_MENU': '619693',    # работает и просто при передаче самого айдишника
    #'EQ_MENU': 'Prevotella sp. oral taxon 472 str. F0295',    # а так не работает
    'NUM_ORG': 1,
    'MAX_NUM_SEQ': 100,
    'EXPECT': 0.05,
    'WORD_SIZE': 6,
    'HSP_RANGE_MAX': 0,
    'MATRIX_NAME': 'BLOSUM62',
    'MATCH_SCORES': [1, -2],
    'GAPCOSTS': '11 1',
    'COMPOSITION_BASED_STATISTICS': 2,
    'FILTER': 'L',
    'REPEATS': 566037,
    'TEMPLATE_LENGTH': 0,
    'TEMPLATE_TYPE': 0,
    'PSSM': '(binary)',
    'SHOW_OVERVIEW': 'true',
    'SHOW_LINKOUT': 'true',
    'GET_SEQUENCE': 'true',
    'FORMAT_OBJECT': 'Alignment',
    'FORMAT_TYPE': 'HTML',
    'ALIGNMENT_VIEW': 'Pairwise',
    'MASK_CHAR': 2,
    'MASK_COLOR': 1,
    'DESCRIPTIONS': 100,
    'ALIGNMENTS': 100,
    'LINE_LENGTH': 60,
    'NEW_VIEW': 'true',
    'NCBI_GI': 'false',
    'SHOW_CDS_FEATURE': 'false',
    'NUM_OVERVIEW': 100,
    'FORMAT_NUM_ORG': 1,
    'CONFIG_DESCR': '2,3,4,5,8,9,10,11,12,13,14',
    'CLIENT': 'web',
    'SERVICE': 'plain',
    'CMD': 'request',
    'PAGE': 'Translations',
    'PROGRAM': 'tblastn',
    'UNGAPPED_ALIGNMENT': 'no',
    'BLAST_PROGRAMS': 'tblastn',
    'DB_DISPLAY_NAME': 'wgs',
    'ORG_DBS': 'orgDbsOnly_wgs',
    'SHOW_ORGANISMS': 'on',
    'SELECTED_PROG_TYPE': 'tblastn',
    'SAVED_SEARCH': 'true',
    'NUM_DIFFS': 1,
    'NUM_OPTS_DIFFS': 0,
    #'UNIQ_DEFAULTS_NAME': 'A_SearchDefaults_1nRG8I_3Rxm_degTm11k3CN_GTXQl_1Twy123r',
    'PAGE_TYPE': 'BlastSearch',
    #'USER_DEFAULT_PROG_TYPE': 'tblastn'
}

In [346]:
prev_time = time.time()

resp = requests.post(BLAST_URL, data=payload)

soup = BeautifulSoup(resp.content)
resp.status_code

200

In [347]:
# Extract RID
RID = soup.find('input', {'name': 'RID'}).get('value')
RID

'2H0DZA6P01R'

## запрашиваем результат по риду, пока не получим его

You may check the status of the search by invoking blast.cgi with the parameters CMD=Get, FORMAT_OBJECT=SearchInfo, and RID=VALUE, where VALUE is the RID returned by the server. The server will respond with a page containing a QBlastInfo block with the status set as one of WAITING, UNKNOWN, or READY.


Once the search is finished, you may retrieve results by invoking blast.cgi with the parameters CMD=GET and RID=VALUE, where VALUE is the RID the system returned. An example would be “CMD=Get&RID=6NTRF1YLO8”. You may also specify the report type (e.g, add FORMAT_TYPE=Text).



In [385]:
payload = {
    'CMD': 'Get',
    'RID': RID,
    'FORMAT_TYPE': 'HTML'   
}

In [386]:
# stop when it takes more than 15 minutes.

delay = 10
wait = 10

i = 0

while True:
    
    current_time = time.time()
    wait = prev_time + delay - current_time
    
    if wait > 0:
        time.sleep(wait)
        prev_time = current_time + wait
        
    if i * delay >= 900:
        print('Oops! Try to check your resalts later (or restart)')
    
    else:
        prev_time = current_time
    
    
    try:
        resp = requests.post(BLAST_URL, data=payload)
        soup = BeautifulSoup(resp.content)
    
    except (ValueError, 
            urllib3.exceptions.InvalidChunkLength, 
            urllib3.exceptions.ProtocolError, 
            requests.exceptions.ChunkedEncodingError):
        print('Please wait. Searching.')
        i += 1
        continue
    
    else:
        if not soup.find('table', {'id': 'statInfo'}):
            print(f'status code: {resp.status_code}, DONE.')
            break
    

status code: 200, DONE.


In [387]:
def save_content(resp, filename):
    with open(filename, "wb") as file:
        file.write(resp.content)
        
save_content(resp, "page2.html")

In [447]:
database = re.match('.*\((.*)\)', soup.find(id="dbName").text).group(1)

In [453]:
seq_list = []

for line in soup.find_all('form', attrs={"id": "formBlastDescr"})[0].find_all('input', attrs={'type': 'checkbox'}):
    seq_list.append('gb|' + re.sub('Select seq ', '', line.text).split('.')[0] +'|')
    
seq_list

['gb|ACZS01000003|',
 'gb|ACZS01000174|',
 'gb|ACZS01000173|',
 'gb|ACZS01000087|']

In [458]:
# get subj_id:

subj_id_list = []

for line in soup.find_all('form', attrs={"id": "formBlastDescr"})[0].find_all('input', attrs={'type': 'checkbox'}):
    subj_id_list.append(re.sub('Select seq ', '', line.text))
    
subj_id_list

['ACZS01000003.1', 'ACZS01000174.1', 'ACZS01000173.1', 'ACZS01000087.1']

In [463]:
soup.find_all('form', attrs={"id": "formBlastDescr"})[0].find_all('input')

[<input id="Rid" type="hidden" value="2H0DZA6P01R"/>,
 <input id="useAlignDB" name="useAlignDB" type="hidden" value=""/>,
 <input id="dbType" type="hidden" value="GenBank"/>,
 <input id="serviceType" type="hidden" value="wgs"/>,
 <input id="clientType" type="hidden" value="web"/>,
 <input id="currQuery" type="hidden" value="lcl|Query_54029"/>,
 <input id="minDispAlnLen" type="hidden" value="2000"/>,
 <input id="maxDispAlnLen" type="hidden" value="20000"/>,
 <input id="entrezLimit" type="hidden" value=""/>,
 <input id="queryIndex" type="hidden" value="0"/>,
 <input id="filteredByAccType" type="hidden" value=""/>,
 <input defval="2,3,6,7,8,9,10,11,12" id="cfcDsInf" type="hidden" value="2,3,6,7,8,9,10,11,12">
 <input id="checkedSeqs" name="CHECKED_SEQS" type="hidden" value="">
 <!-- The following is used for on screen sorting -->
 <input id="sortedDescrSeqs" name="SORTED_DESCR_SEQS" type="hidden" value="">
 <input id="sortBy" name="SORT_BY" type="hidden" value="">
 <input id="displaySort"

In [454]:
def download_fasta_algnmt(RID, database, seq_list, filename):
    
    FASTA_URL = 'https://www.ncbi.nlm.nih.gov/blast/SEQDUMP_SRA/seqdump.cgi'
    align_seq_list = ','.join(seq_list)
    
    payload = {
            'DATABASE': database,
            'RID': RID,
            'SERVICE_TYPE': 'wgs',
            'DBTYPE': 'nucl',
            'DOWNLOAD_TYPE': 'range',
            'ALIGN_SEQ_LIST': align_seq_list
    }
    
    resp = requests.post(FASTA_URL, data=payload)
    
    with open(filename, "wb") as file:
        file.write(resp.content)

In [455]:
filename = "page2.fa"
download_fasta_algnmt(RID, database, seq_list, filename)

## Приводим результаты в читаемый вид

In [388]:
for header in resp.headers:
    print(header, '\t', resp.headers[header])

Date 	 Wed, 09 Mar 2022 12:44:39 GMT
Server 	 Apache
Strict-Transport-Security 	 max-age=31536000; includeSubDomains; preload
Referrer-Policy 	 origin-when-cross-origin
Content-Security-Policy 	 upgrade-insecure-requests
Cache-Control 	 private
NCBI-PHID 	 74C80D94228A13710000000000000001.m_2
NCBI-SID 	 74C80D94228A1371_0000SID
Content-Type 	 text/html
Set-Cookie 	 BlastCubbyImported=passive; domain=ncbi.nlm.nih.gov, MyBlastUser=1JS6BBUzD2DEVX05Z1E225665; domain=.ncbi.nlm.nih.gov; path=/, ncbi_sid=74C80D94228A1371_0000SID; domain=.nih.gov; path=/; expires=Thu, 09 Mar 2023 12:44:39 GMT
X-UA-Compatible 	 IE=Edge
X-XSS-Protection 	 1; mode=block
Keep-Alive 	 timeout=1, max=10
Connection 	 Keep-Alive
Transfer-Encoding 	 chunked


In [266]:
for link in soup.find_all('a'):
    print(link.get('href'))

https://www.ncbi.nlm.nih.gov/
https://www.ncbi.nlm.nih.gov/myncbi
#mainCont
#navcontent
https://www.ncbi.nlm.nih.gov/guide/browsers/#accesskeys
https://www.ncbi.nlm.nih.gov/
https://www.ncbi.nlm.nih.gov/account/?back_url=https%3A%2F%2Fblast%2Encbi%2Enlm%2Enih%2Egov%2FBlast%2Ecgi%3FADV%5FVIEW%3Dtrue%26ALIGNMENTS%3D50%26ALIGNMENT%5FVIEW%3DPairwise%26BLAST%5FPROGRAMS%3Dtblastn%26CMD%3DGet%26COMPOSITION%5FBASED%5FSTATISTICS%3D2%26DATABASE%3DWGS%5FVDB%253A%252F%252FACZS01%26DB%5FDISPLAY%5FNAME%3Dwgs%24619693%253B%26DB%5FGROUP%3DwgsOrg%26DESCRIPTIONS%3D100%26DYNAMIC%5FFORMAT%3Don%26ENTREZ%5FQUERY%3D%26EQ%5FMENU%3D619693%26EXPECT%3D0%2E05%26FILTER%3DL%26FORMAT%5FOBJECT%3DAlignment%26FORMAT%5FTYPE%3DHTML%26GAPCOSTS%3D11%2B1%26GET%5FSEQUENCE%3Dtrue%26HSP%5FRANGE%5FMAX%3D0%26JOB%5FTITLE%3Dfasta%26LINE%5FLENGTH%3D60%26MASK%5FCHAR%3D2%26MASK%5FCOLOR%3D1%26MATRIX%5FNAME%3DBLOSUM62%26MAX%5FNUM%5FSEQ%3D100%26MYNCBI%5FUSER%3D11557132753%26MYNCBI%5FUSER%3D11557132753%26MYNCBI%5FUSER%3D11557132703%26NCB

In [ ]:
class TBlastN():
    '''
    Class for search in tblastn database.
    '''
    
    def __init__(self):
        
        
        
    def tn_search(self, sequence, database='WGS', organism):
        '''
        функцию для запроса, которая будет принимать 3 обязательных аргумента: 
        белковая последовательность, которую мы бластим, 
        базу данных (в этом задании нас интересует только WGS, но по желанию можете добавить что-нибудь ещё), 
        таксон, у которого мы ищем последовательность, чаще всего - конкретный вид.
        
        Функция дожна возвращать список объектов типа Alignment, у него должны быть следующие атрибуты
        self.score
        self.subj_range    # tuple
        self.subj_name
        self.subj_id
        self.subj_len
        self.e_value
        self.identity
        self.query_seq
        self.subj_seq
        
        '''
        
        return self.alingment
    
    
    
    def get_aligment(self):
        
        self.score
        self.subj_range    # tuple
        self.subj_name
        self.subj_id
        self.subj_len
        self.e_value
        self.identity
        self.query_seq
        self.subj_seq
        
    